## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tensorflow import keras

from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense, SimpleRNN, GRU
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

## Load Data

In [ ]:
google_training_complete = pd.read_csv("/kaggle/input/google-stock-2010-2023/Google_Stock_Train (2010-2022).csv")

# Convert 'Date' column to datetime format
google_training_complete['Date'] = pd.to_datetime(google_training_complete['Date'])

google_training_complete.head(10)

## EDA

In [ ]:
# Line chart for Google stock price over time
fig1 = px.line(google_training_complete, x='Date', y='Close', title='Google Stock Price Over Time')
fig1.show()

In [ ]:
# Scatter plot of daily trading volume
fig2 = px.scatter(google_training_complete, x='Date', y='Volume', title='Daily Trading Volume')
fig2.show()

In [ ]:
# Box plot of Google stock prices for each year
google_training_complete['Year'] = google_training_complete['Date'].dt.year
fig3 = px.box(google_training_complete, x='Year', y='Close', title='Google Stock Prices - Yearly Box Plot')
fig3.show()

In [ ]:
# Candlestick chart for stock prices
fig4 = go.Figure(data=[go.Candlestick(x=google_training_complete['Date'],
                                     open=google_training_complete['Open'],
                                     high=google_training_complete['High'],
                                     low=google_training_complete['Low'],
                                     close=google_training_complete['Close'])])

fig4.update_layout(title='Google Stock Prices - Candlestick Chart',
                   xaxis_title='Date',
                   yaxis_title='Stock Price')

fig4.show()

In [ ]:
# Histogram of daily returns
google_training_complete['Daily_Return'] = google_training_complete['Close'].pct_change()
fig5 = px.histogram(google_training_complete, x='Daily_Return', nbins=30, title='Distribution of Daily Returns')
fig5.show()

In [ ]:
# Heatmap of correlation matrix
correlation_matrix = google_training_complete.corr()
fig6 = px.imshow(correlation_matrix, x=correlation_matrix.index, y=correlation_matrix.columns, title='Correlation Matrix Heatmap')
fig6.show()

In [ ]:
# Moving average of closing prices
google_training_complete['MA_50'] = google_training_complete['Close'].rolling(window=50).mean()
fig7 = px.line(google_training_complete, x='Date', y=['Close', 'MA_50'], title='Google Stock Price with 50-Day Moving Average')
fig7.show()

In [ ]:
# Scatter plot of closing prices vs. trading volume
fig8 = px.scatter(google_training_complete, x='Close', y='Volume', title='Closing Prices vs. Trading Volume')
fig8.show()

In [ ]:
# Line chart for daily stock price change
google_training_complete['Daily_Change'] = google_training_complete['Close'].diff()
fig9 = px.line(google_training_complete, x='Date', y='Daily_Change', title='Daily Stock Price Change')
fig9.show()

In [ ]:
# Bar chart of trading volume per month
google_training_complete['Month'] = google_training_complete['Date'].dt.month
monthly_volume = google_training_complete.groupby('Month')['Volume'].sum().reset_index()
fig10 = px.bar(monthly_volume, x='Month', y='Volume', title='Total Trading Volume per Month')
fig10.show()

In [ ]:
# Area chart for daily trading volume
fig11 = px.area(google_training_complete, x='Date', y='Volume', title='Daily Trading Volume (Area Chart)')
fig11.show()

In [ ]:
# Histogram of daily stock price changes
fig12 = px.histogram(google_training_complete, x='Daily_Change', title='Distribution of Daily Stock Price Changes')
fig12.show()

In [ ]:
# Line chart for daily stock price percent change
google_training_complete['Daily_Pct_Change'] = google_training_complete['Close'].pct_change() * 100
fig13 = px.line(google_training_complete, x='Date', y='Daily_Pct_Change', title='Daily Stock Price Percent Change')
fig13.show()

In [ ]:
# Box plot of daily stock price percent change by month
fig14 = px.box(google_training_complete, x='Month', y='Daily_Pct_Change', title='Daily Stock Price Percent Change by Month')
fig14.show()

In [ ]:
# Scatter plot of closing prices with trendline
fig15 = px.scatter(google_training_complete, x='Date', y='Close', title='Google Stock Prices with Trendline', trendline='lowess')
fig15.show()

In [ ]:
# Line chart for daily trading volume with moving average
google_training_complete['MA_Volume'] = google_training_complete['Volume'].rolling(window=10).mean()
fig16 = px.line(google_training_complete, x='Date', y=['Volume', 'MA_Volume'], title='Daily Trading Volume with 10-Day Moving Average')
fig16.show()

In [ ]:
# Box plot of daily stock price changes by year
fig17 = px.box(google_training_complete, x='Year', y='Daily_Change', title='Daily Stock Price Changes by Year')
fig17.show()

In [ ]:
# Line chart for daily closing prices in 2022
df_2022 = google_training_complete[google_training_complete['Year'] == 2022]
fig18 = px.line(google_training_complete, x='Date', y='Close', title='Google Stock Prices in 2022')
fig18.show()

In [ ]:
# Scatter plot of daily returns vs. trading volume
fig19 = px.scatter(google_training_complete, x='Daily_Return', y='Volume', title='Daily Returns vs. Trading Volume')
fig19.show()

In [ ]:
# Line chart for daily trading volume with range slider
fig20 = px.line(google_training_complete, x='Date', y='Volume', title='Daily Trading Volume with Range Slider')
fig20.update_xaxes(rangeslider_visible=True)
fig20.show()

## Data preprocessing

In [ ]:
google_training_processed = google_training_complete.iloc[:, 4:5].values
google_training_processed

In [ ]:
scaler = MinMaxScaler(feature_range = (0, 1))

google_training_scaled = scaler.fit_transform(google_training_processed)

In [ ]:
print(google_training_scaled)
print(google_training_scaled.shape)

In [ ]:
features_set = []
labels = []
for i in range(60, google_training_scaled.shape[0]):
    features_set.append(google_training_scaled[i-60:i, 0])
    labels.append(google_training_scaled[i, 0])

In [ ]:
type(features_set)

In [ ]:
features_set, labels = np.array(features_set), np.array(labels)

In [ ]:
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))
print(features_set.shape)

In [ ]:
RNN_model = Sequential()
RNN_model.add(SimpleRNN(units=300, return_sequences=True, input_shape=(features_set.shape[1], 1)))
RNN_model.add(Dropout(0.2))

RNN_model.add(SimpleRNN(units=100, return_sequences=True))
RNN_model.add(Dropout(0.2))

RNN_model.add(SimpleRNN(units=100, return_sequences=True))
RNN_model.add(Dropout(0.2))

RNN_model.add(SimpleRNN(units=100))
RNN_model.add(Dropout(0.2))

RNN_model.add(Dense(units = 1))

In [ ]:
RNN_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
RNN_model.summary()

In [ ]:
RNN_History = RNN_model.fit(features_set, labels, epochs = 300, batch_size = 32)

In [ ]:
LSTM_model = Sequential()
LSTM_model.add(LSTM(units=300, return_sequences=True, input_shape=(features_set.shape[1], 1)))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(LSTM(units=100, return_sequences=True))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(LSTM(units=100, return_sequences=True))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(LSTM(units=100))
LSTM_model.add(Dropout(0.2))

LSTM_model.add(Dense(units = 1))

In [ ]:
LSTM_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
LSTM_model.summary()

In [ ]:
LSTM_History = LSTM_model.fit(features_set, labels, epochs = 300, batch_size = 32)

In [ ]:
GRU_model = Sequential()
GRU_model.add(GRU(units=300, return_sequences=True, input_shape=(features_set.shape[1], 1)))
GRU_model.add(Dropout(0.2))

GRU_model.add(GRU(units=100, return_sequences=True))
GRU_model.add(Dropout(0.2))

GRU_model.add(GRU(units=100, return_sequences=True))
GRU_model.add(Dropout(0.2))

GRU_model.add(GRU(units=100))
GRU_model.add(Dropout(0.2))

GRU_model.add(Dense(units = 1))

In [ ]:
GRU_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
GRU_model.summary()

In [ ]:
GRU_History = GRU_model.fit(features_set, labels, epochs = 300, batch_size = 32)

In [ ]:
fig = go.Figure()

# Training loss trace
fig.add_trace(go.Scatter(x=list(range(1, len(RNN_History.history['loss']) + 1)),
                         y=RNN_History.history['loss'],
                         mode='lines',
                         name='RNN Training Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(LSTM_History.history['loss']) + 1)),
                         y=LSTM_History.history['loss'],
                         mode='lines',
                         name='LSTM Training Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(GRU_History.history['loss']) + 1)),
                         y=GRU_History.history['loss'],
                         mode='lines',
                         name='GRU Training Loss'))

fig.update_layout(title='Training and Validation Loss',
                  xaxis_title='Epoch',
                  yaxis_title='Loss',
                  legend_title='Loss Type',
                  hovermode='x',
                  hoverlabel=dict(bgcolor='white', font_size=12, font_family='Rockwell'),
                  template='plotly_white')

fig.show()

#### Evalute Model Prediction

In [ ]:
google_testing_complete = pd.read_csv("/kaggle/input/google-stock-2010-2023/Google_Stock_Test (2023).csv")
google_testing_processed = google_testing_complete.iloc[:, 1:2].values

google_total = pd.concat((google_training_complete['Close'], google_testing_complete['Close']), axis=0)

test_inputs = google_total[len(google_total) - len(google_testing_complete) - 60:].values
test_inputs

In [ ]:
test_inputs = test_inputs.reshape(-1,1)
test_inputs = scaler.transform(test_inputs)

test_features = []
for i in range(60, len(test_inputs)):
    test_features.append(test_inputs[i-60:i, 0])

test_features = np.array(test_features)
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1))
test_features.shape

In [ ]:
# Evaluate the model on the test set
## RNN
RNN_test_loss = RNN_model.evaluate(test_features, google_testing_processed)
print(f'RNN Test Loss: {RNN_test_loss}')

## LSTM
LSTM_test_loss = LSTM_model.evaluate(test_features, google_testing_processed)
print(f'LSTM Test Loss: {LSTM_test_loss}')

## GRU
GRU_test_loss = GRU_model.evaluate(test_features, google_testing_processed)
print(f'LSTM Test Loss: {GRU_test_loss}')

## Make predictions

In [ ]:
# RNN
RNN_predictions = RNN_model.predict(test_features)
RNN_predictions = scaler.inverse_transform(RNN_predictions)

# LSTM
LSTM_predictions = LSTM_model.predict(test_features)
LSTM_predictions = scaler.inverse_transform(LSTM_predictions)

# GRU
GRU_predictions = GRU_model.predict(test_features)
GRU_predictions = scaler.inverse_transform(GRU_predictions)

In [ ]:
google_testing_complete

In [ ]:
google_prediction = google_testing_complete['Date']
google_prediction = pd.DataFrame(google_prediction)
google_prediction['RNN_Open'] = RNN_predictions
google_prediction['LSTM_Open'] = LSTM_predictions
google_prediction['GRU_Open'] = GRU_predictions

google_prediction

In [ ]:
fig = go.Figure()

# Actual Apple stock price trace
fig.add_trace(go.Scatter(x=google_testing_complete.Date,
                         y=google_testing_complete.Open,
                         mode='lines',
                         name='Actual Google Stock Price',
                         line=dict(color='blue')))

# RNN Predicted Apple stock price trace
fig.add_trace(go.Scatter(x=google_prediction.Date,
                         y=google_prediction.RNN_Open,
                         mode='lines',
                         name='RNN Predicted Google Stock Price',
                         line=dict(color='red')))

# LSTM Predicted Apple stock price trace
fig.add_trace(go.Scatter(x=google_prediction.Date,
                         y=google_prediction.LSTM_Open,
                         mode='lines',
                         name='LSTM Predicted Google Stock Price',
                         line=dict(color='green')))

# LSTM Predicted Apple stock price trace
fig.add_trace(go.Scatter(x=google_prediction.Date,
                         y=google_prediction.GRU_Open,
                         mode='lines',
                         name='GRU Predicted Google Stock Price',
                         line=dict(color='yellow')))

fig.update_layout(title='Google Stock Price Prediction',
                  xaxis_title='Date',
                  yaxis_title='Google Stock Price',
                  legend_title='Price Type',
                  hovermode='x',
                  hoverlabel=dict(bgcolor='white', font_size=12, font_family='Rockwell'),
                  template='plotly_white')

fig.show()